In [15]:
# | default_exp classes.DomoGroup


In [16]:
# | exporti
from dataclasses import dataclass, field
from typing import List

import httpx

from fastcore.basics import patch_to


# from ..utils.Base import Base
# from ..utils.DictDot import DictDot

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.utils.DictDot as util_dd

import domolibrary.routes.group as group_routes

# import domolibrary.routes.publish as publish_routes


# Group Membership

In [17]:
# | export
class GroupMembership:
    group_members_id_ls: list[str] = None
    group_owners_id_ls: list[str] = None
    group = None

    def __init__(self, group):
        self.group = group
        self._update_group_members_ids()

    def _update_group_members_ids(self, ):
        if self.group.group_members_id_ls and len(self.group.group_members_id_ls) > 0:
            # print(f"_update_group_members_ids: {len(self.group.group_members)}")

            self.group_members_id_ls = self.group.group_members_id_ls

            return self.group_members_id_ls

    async def _update_group_access(
        self,
        auth: dmda.DomoAuth,
        body,  # use group_routes.generate_body_update_group_membership to generate an appropriate body
        return_raw: bool = False,
        debug_api: bool = False,
        session: httpx.AsyncClient = None,
    ):

        res = await group_routes.update_group_membership(
            auth=auth, body=body, debug_api=debug_api, session=session
        )

        if return_raw:
            return res

        return res


In [18]:
# | exporti

@patch_to(GroupMembership)
async def set_membership(
    self: GroupMembership,
    user_ids_list : list[str],
    auth: dmda.DomoAuth = None,
    return_raw: bool = False,
    debug_api: bool = False, session: httpx.AsyncClient = None,
):

    auth = auth or self.group.auth

    add_user_arr = [str(uid) for uid in user_ids_list]

    remove_user_arr = [
        str(uid) for uid in self.group_members_id_ls if str(uid) not in add_user_arr
    ]

    body = group_routes.generate_body_update_group_membership(
        group_id=self.group.id,
        add_user_arr=add_user_arr,
        remove_user_arr=remove_user_arr,
    )

    return await self._update_group_access(
        auth=auth, body=body, debug_api=debug_api, session=session, return_raw = return_raw
    )

In [19]:
# | exporti
@patch_to(GroupMembership)
async def alter_owner(self: GroupMembership, auth: dmda.DomoAuth,
                      add_owner_ids_ls=None,
                      remove_owner_ids_ls=None,
                      debug_api: bool = False,
                      session: httpx.AsyncClient = None
                      ):

    body = group_routes.generate_body_update_group_membership(group_id=self.group.id,
                                                              add_owner_user_arr=add_owner_ids_ls,
                                                              remove_owner_user_arr=remove_owner_ids_ls)

    return await self._update_group_access(auth=auth, body=body, log_results=log_results, debug_api=debug)


In [20]:

#     async def alter_membership(self, auth: dmda.DomoAuth, add_user_ids_list, remove_users_ids_list,
#                                log_results: bool = False, debug_api: bool = False):

#         body = group_routes.generate_body_update_group_membership(group_id=self.group.id,
#                                                                   add_user_arr=add_user_ids_list,
#                                                                   remove_user_arr=remove_users_ids_list)

#         return await self._update_group_access(auth=auth, body=body, log_results=log_results, debug_api=debug)

# Domo Group

In [38]:
# | export
@dataclass
class DomoGroup:
    auth: dmda.DomoAuth = field(repr=False, default=None)
    id: str = None
    name: str = None
    type: str = None
    description: str = None
    group_members_id_ls: list[str] = field(repr=False, default_factory=list)
    owner_members_id_ls: list[str] = field(repr=False, default_factory=list)
    
    group_members_ls: list[dict] = field(repr=False, default_factory=list)
    owner_members_ls : list[dict]= field(repr = False, default_factory = list)

    def __post_init__(self):
        # self.domo_instance = self.domo_instance or auth.domo_instance
        self.Membership = GroupMembership(self)

    @classmethod
    def _from_group_json(cls, auth: dmda.DomoAuth, json_obj):
        dd = json_obj

        if not isinstance(json_obj, util_dd.DictDot):
            dd = util_dd.DictDot(json_obj)

        return cls(
            auth=auth,
            id=dd.id,
            name=dd.name,
            description=dd.description,
            type=dd.type or dd.groupType,
            group_members_id_ls=ddd.userIds,
            owner_members=dd.owners,
        )

    @classmethod
    def _from_grouplist_json(cls, auth: dmda.DomoAuth, json_obj):
        dd = json_obj

        if not isinstance(json_obj, util_dd.DictDot):
            dd = util_dd.DictDot(json_obj)

        return cls(
            auth=auth,
            id=dd.groupId,
            name=dd.name,
            description=dd.description,
            type=dd.groupType,
            owner_members_ls = dd.owners,
            owner_members_id_ls = [owner.id for owner in dd.owners],
            
            group_members_ls=dd.groupMembers,
            group_members_id_ls= [member.id for member in dd.members]
        )

    @staticmethod
    def _groups_to_domo_group(json_list, auth: dmda.DomoAuth) -> List[dict]:
        domo_groups = [
            DomoGroup._from_search_json(auth=auth, json_obj=json_obj)
            for json_obj in json_list
        ]

        return domo_groups

TypeError: 'owner_members_ls' is a field but has no type annotation

In [22]:
# | exporti
@patch_to(DomoGroup, cls_method=True)
async def get_by_id(
    cls,
    auth: dmda.DomoAuth,
    group_id: str,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):

    res = await group_routes.get_group_by_id(
        auth=auth, group_id=group_id, debug_api=debug_api, session=session
    )
    if return_raw:
        return res

    if res.status == 200:
        return cls._from_groups_API_json(auth=auth, json_obj=res.response)

#### sample implementation of get_by_id


In [23]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 2146122228

await DomoGroup.get_by_id(auth=token_auth, group_id=group_id, return_raw= False)

DomoGroup(id=2146122228, name='test', type='closed', description='with_description')

#### sample implementation of Group Membership
##### set_membership

In [24]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 2146122228

membership = [1141078945, 1725780094, 861444808, 1802840904]

domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(f"validate membership {', '.join([str(id) for id in domo_group.Membership.group_members_id_ls])}")

await domo_group.Membership.set_membership(
    user_ids_list= membership
)


validate membership 1141078945, 1725780094, 1802840904, 861444808


ResponseGetData(status=200, response='', is_success=True)

##### alter_owners

In [25]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 2146122228

add_membership = [1141078945, 1725780094, 861444808, 1802840904]

domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(f"validate membership {', '.join([str(id) for id in domo_group.Membership.group_members_id_ls])}")

await domo_group.Membership.set_membership(
    user_ids_list= membership, return_raw = True
)


validate membership 1141078945, 1725780094, 1802840904, 861444808


ResponseGetData(status=200, response='', is_success=True)

In [26]:
# import os

# token_auth = dmda.DomoTokenAuth(
#     domo_instance="domo-community",
#     domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
# )

# group_id = 2146122228

# membership = [1141078945, 1725780094, 861444808, 1802840904]

# domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

# print(
#     f"validate membership {', '.join([str(id) for id in domo_group.Membership.group_members_id_ls])}")

# await domo_group.Membership.set_membership(
#     user_ids_list=membership
# )


In [27]:
# | exporti

@patch_to(DomoGroup, cls_method=True)
async def create_from_name(
    cls: DomoGroup,
    auth: dmda.DomoAuth,
    group_name: str = None,
    group_type: str = "open",  # use GroupType_Enum
    description: str = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):

    res = await group_routes.create_group(
        auth=auth,
        group_name=group_name,
        group_type=group_type,
        description=description,
        debug_api=debug_api,
    )

    domo_group = cls._from_search_json(auth=auth, json_obj=res.response)


#### sample implementation of create_from_name


In [28]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "Test Group 2"
try:
    await DomoGroup.create_from_name(group_name=group_name, auth=token_auth)

except group_routes.CreateGroup_Error as e:
    print(e)

create_group: Status 400 - Test Group 2 already exists. Choose a different group_name at domo-community


In [29]:
# | exporti


@patch_to(DomoGroup, cls_method=True)
async def search_by_name(
    cls,
    auth: dmda.DomoAuth,
    group_name: str,
    is_exact_match: bool = True,
    create_if_not_exist: bool = True,  # will set is_exact_match = True if enabled
    debug_api: bool = False,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):

    if create_if_not_exist:
        is_exact_match = True

    try:
        res = await group_routes.search_groups_by_name(
            auth=auth,
            search_name=group_name,
            debug_api=debug_api,
            is_exact_match=is_exact_match,
            session=session,
        )
        json_list = res.response

        if isinstance(json_list, list):
            return cls._groups_to_domo_group(json_list, auth)

        return cls._from_search_json(auth=auth, json_obj=json_list)

    except group_routes.SearchGroups_Error as e:
        print("create group")

        return await DomoGroup.create_from_name(
            auth=auth, group_name=group_name, debug_api=debug_api, session=session
        )


#### sample implementation of search_by_name


In [30]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "Test Group 2"

res = await DomoGroup.search_by_name(group_name=group_name, auth=token_auth)
print(res)

DomoGroup(id=1259653287, name='Test Group 2', type='open', description='')


# Domo Groups

In [31]:
# | export
class DomoGroups:
    def __init__(self):
        pass

    @staticmethod
    def _groups_to_domo_group(json_list, auth: dmda.DomoAuth):

        return [
            DomoGroup._from_search_json(auth=auth, json_obj=json_obj)
            for json_obj in json_list
        ]

In [32]:
# | export
@patch_to(DomoGroups, cls_method=True)
async def get_all_groups(
    cls: DomoGroups,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):

    res = await group_routes.get_all_groups(
        auth=auth, debug_api=debug_api, session=session
    )

    if len(res.response) > 0:
        json_list = res.response

        return cls._groups_to_domo_group(json_list=json_list, auth=auth)

    else:
        return []


#### sample implementation of get_all_groups


In [33]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_groups = await DomoGroups.get_all_groups(auth=token_auth)
domo_groups[0:5]


[DomoGroup(id=1814479647, name='Admin Test', type='closed', description=''),
 DomoGroup(id=926594289, name='April 2023 Cohort', type='closed', description='Group for ALL Domo Customer Orientation workshop attendees in April 2023'),
 DomoGroup(id=180920085, name='Content', type='adHoc', description=None),
 DomoGroup(id=892007656, name='DCO Team Guides', type='closed', description='Admin Group - for Domo Customer Orientation workshops'),
 DomoGroup(id=1324037627, name='Default', type='adHoc', description=None)]